* [#42](https://github.com/salgo60/SamlaLibris/issues/42) 
* denna Notebook [RAA42_linkroot.ipynb](https://github.com/salgo60/SamlaLibris/blob/master/notebook/RAA42_linkroot.ipynb)
   *  [links_raa_2025_11_30.html](https://salgo60.github.io/SamlaLibris/notebook/results/links_raa_2025_11_30.html) 

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-12-01 18:26:50


In [2]:
SCRIPT_NAME = "RAA42_linkroot.ipynb"
SCRIPT_URL = (
    "https://github.com/salgo60/SamlaLibris/"
    "blob/master/Notebook/RAA42_linkroot.ipynb"
)

In [3]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)



Current Working Directory: /Users/salgo/Documents/GitHub/SamlaLibris/notebook


In [4]:
def read_domains(file_path):
    print(f"[DEBUG] Reading domains from: {file_path}")
    print(f"[DEBUG] Current Working Directory: {current_directory}")

    df = pd.read_csv(file_path, header=0)   # <- skip header row
    domains_list = df.iloc[:, 0].dropna().unique().tolist()
    print(f"[DEBUG] Found {len(domains_list)} domains.")
    return domains_list


In [5]:
import requests

def fetch_sitematrix_df():
    url = "https://meta.wikimedia.org/w/api.php"
    params = {
        "action": "sitematrix",
        "format": "json"
    }
    headers = {
        "User-Agent": "salgo60-language-fetcher/1.0 (salgo60@msn.com)"
    }

    print("[DEBUG] Fetching sitematrix…")
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()

    if "application/json" not in r.headers.get("Content-Type", ""):
        raise ValueError("Server returned non-JSON response")

    data = r.json()["sitematrix"]

    rows = []

    # --- language-specific sites ---
    for key, lang_block in data.items():
        if not key.isdigit():
            continue  # skip "count", "specials"

        lang_code = lang_block.get("code")
        lang_name = lang_block.get("name")

        for site in lang_block.get("site", []):
            rows.append({
                "lang_code": lang_code,
                "lang_name": lang_name,
                "project": site.get("project"),
                "url": site.get("url"),
                "dbname": site.get("dbname"),
                "site_name": site.get("sitename"),
                "closed": site.get("closed", False)
            })

    # --- special wikis (Wikidata, Commons, Meta, etc.) ---
    for site in data.get("specials", []):
        rows.append({
            "lang_code": "special",
            "lang_name": "special",
            "project": site.get("project"),
            "url": site.get("url"),
            "dbname": site.get("dbname"),
            "site_name": site.get("sitename"),
            "closed": site.get("closed", False)
        })

    return pd.DataFrame(rows)


In [6]:
import requests
import pandas as pd


HEADERS = {
    "User-Agent": "salgo60-language-fetcher/2.0 (https://github.com/salgo60)"
}


df_wiki = fetch_sitematrix_df()
df_wiki["closed"] = df_wiki["closed"].fillna(False).astype(bool)
df_wiki = df_wiki[df_wiki["site_name"] == "Wikipedia"]
df_wiki.head()



[DEBUG] Fetching sitematrix…


,lang_code,lang_name,project,url,dbname,site_name,closed
0,aa,Qafár af,None,https://aa.wikipedia.org,aawiki,Wikipedia,False
5,ace,Acèh,None,https://ace.wikipedia.org,acewiki,Wikipedia,False
7,af,Afrikaans,None,https://af.wikipedia.org,afwiki,Wikipedia,False
11,ak,None,None,https://ak.wikipedia.org,akwiki,Wikipedia,False
18,ami,Pangcah,None,https://ami.wikipedia.org,amiwiki,Wikipedia,False


In [7]:
df_wiki.info()

<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, 0 to 1061
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lang_code  207 non-null    object
 1   lang_name  206 non-null    object
 2   project    0 non-null      object
 3   url        207 non-null    object
 4   dbname     207 non-null    object
 5   site_name  207 non-null    object
 6   closed     207 non-null    bool  
dtypes: bool(1), object(6)
memory usage: 11.5+ KB


In [8]:
len(df_wiki)

207

In [9]:
# -----------------------------------------------------------
# Fetch exturlusage entries for one lang/domain
# -----------------------------------------------------------
def fetch_exturlusage(lang, domain):
    base = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "exturlusage",
        "euquery": domain,
        "eulimit": "max"
    }
    while True:
        r = session.get(base, params=params, timeout=10)
        try:
            data = r.json()
        except ValueError:
            print(f"[WARN] {lang}: JSON decode failed")
            break

        for item in data.get("query", {}).get("exturlusage", []):
            yield {
                "lang": lang,
                "page_title": item.get("title"),
                "url": item.get("url"),
                "wiki_link": f"https://{lang}.wikipedia.org/wiki/{item.get('title').replace(' ', '_')}"
            }

        if "continue" not in data:
            break
        params.update(data["continue"])
        time.sleep(0.3)

In [10]:
import os
import time
import random
import requests
import pandas as pd
from urllib.parse import urlparse
from tqdm.notebook import tqdm
file_path_domain = "sources/domains.csv"
domains = read_domains(file_path_domain)
print(domains)


[DEBUG] Reading domains from: sources/domains.csv
[DEBUG] Current Working Directory: /Users/salgo/Documents/GitHub/SamlaLibris/notebook
[DEBUG] Found 11 domains.
['raa.se', 'kringla.nu', 'fornsok.se', 'kulturnav.org', 'k-samsok.se', 'kulturmiljolagen.se', 'platsr.se', 'minnen.se', 'digisam.se', 'k-blogg.se', 'kulturarvsdata.se']


In [11]:
# -------------------------
# Session & helpers
# -------------------------
session = requests.Session()
session.headers.update(
    {"User-Agent": "RAA-LinkAudit/1.0 salgo60@msn.com"}
)

# Filter out special languages
df_wiki_lang = df_wiki[df_wiki["lang_code"] != "special"]
print("Antal Språk:", len(df_wiki_lang))

results = []

for _, row in df_wiki_lang.iterrows():
    lang = row["lang_code"]
    url = row["url"]
    lang_name = row["lang_name"]

    before = len(results)
    for d in domains:
        #print(lang, url, lang_name, d)
        before_domain = len(results)


        for entry in fetch_exturlusage(lang, d):
            entry["domain"] = d
            entry["lang_code"] = lang
            entry["lang_name"] = lang_name
            results.append(entry)
        added_domain = len(results) - before_domain
        if added_domain > 0:
            print("\t",d, "- nr added domain: ", added_domain)

    added = len(results) - before
    print(lang, lang_name, "- nr added lang: ", added)


Antal Språk: 184
aa Qafár af - nr added lang:  0
	 kulturnav.org - nr added domain:  3
ace Acèh - nr added lang:  3
	 raa.se - nr added domain:  1
	 kulturnav.org - nr added domain:  1117
af Afrikaans - nr added lang:  1118
ak None - nr added lang:  0
ami Pangcah - nr added lang:  0
	 kulturnav.org - nr added domain:  845
	 kulturarvsdata.se - nr added domain:  252
an aragonés - nr added lang:  1097
	 raa.se - nr added domain:  2
	 kulturnav.org - nr added domain:  982
	 kulturarvsdata.se - nr added domain:  410
ast asturianu - nr added lang:  1394
	 kulturnav.org - nr added domain:  11
av авар - nr added lang:  11
avk Kotava - nr added lang:  0
ay Aymar aru - nr added lang:  0
	 kulturnav.org - nr added domain:  1
bar Boarisch - nr added lang:  1
bbc Batak Toba - nr added lang:  0
	 kulturnav.org - nr added domain:  22
bcl Bikol Central - nr added lang:  22
bi Bislama - nr added lang:  0
bm bamanankan - nr added lang:  0
bo བོད་ཡིག - nr added lang:  0
br brezhoneg - nr added lang:  0


In [12]:
#results

In [13]:
domains

['raa.se',
 'kringla.nu',
 'fornsok.se',
 'kulturnav.org',
 'k-samsok.se',
 'kulturmiljolagen.se',
 'platsr.se',
 'minnen.se',
 'digisam.se',
 'k-blogg.se',
 'kulturarvsdata.se']

In [14]:
df_raa = pd.DataFrame(results)

In [15]:
df_raa

,lang,page_title,url,wiki_link,domain,lang_code,lang_name
0,ace,Leonard Cohen,http://kulturnav.org/language/en/7e908441-ad8d...,https://ace.wikipedia.org/wiki/Leonard_Cohen,kulturnav.org,ace,Acèh
1,ace,Agnetha Fältskog,http://kulturnav.org/language/en/1718a400-a2bc...,https://ace.wikipedia.org/wiki/Agnetha_Fältskog,kulturnav.org,ace,Acèh
2,ace,Anni-Frid Lyngstad,http://kulturnav.org/language/en/a75e48ae-e70f...,https://ace.wikipedia.org/wiki/Anni-Frid_Lyngstad,kulturnav.org,ace,Acèh
3,af,Adils,http://www.raa.se/cms/extern/se_och_besoka/sev...,https://af.wikipedia.org/wiki/Adils,raa.se,af,Afrikaans
4,af,Oscar Niemeyer,http://kulturnav.org/6aa00405-f813-4e04-96b2-1...,https://af.wikipedia.org/wiki/Oscar_Niemeyer,kulturnav.org,af,Afrikaans
...,...,...,...,...,...,...,...
305245,zh,日本,http://kulturnav.org/de944dd7-838e-46d0-a342-b...,https://zh.wikipedia.org/wiki/日本,kulturnav.org,zh,中文
305246,zh,約翰·米爾斯 (演員),http://kulturnav.org/04ac3136-7fd5-470d-bb3e-0...,https://zh.wikipedia.org/wiki/約翰·米爾斯_(演員),kulturnav.org,zh,中文
305247,zh,罗伯托·布拉科,http://kulturnav.org/91357fd7-edaa-4ea7-9b6f-7...,https://zh.wikipedia.org/wiki/罗伯托·布拉科,kulturnav.org,zh,中文
305248,zh,KulturNav,http://www.digisam.se/images/docs/Bodin_Kultur...,https://zh.wikipedia.org/wiki/KulturNav,digisam.se,zh,中文


In [16]:
len(df_raa)

305250

In [17]:
df_raa = df_raa.rename(columns={
    "lang": "Språk",
    "page_title": "Wikipedia-sida",
    "url": "Extern länk",
    "wiki_link": "Wikipedia-länk",
    "domain": "Domän"
})

In [18]:
df_raa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305250 entries, 0 to 305249
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Språk           305250 non-null  object
 1   Wikipedia-sida  305250 non-null  object
 2   Extern länk     305250 non-null  object
 3   Wikipedia-länk  305250 non-null  object
 4   Domän           305250 non-null  object
 5   lang_code       305250 non-null  object
 6   lang_name       305250 non-null  object
dtypes: object(7)
memory usage: 16.3+ MB


In [19]:
import pandas as pd

# --- Stats ---
total_links = len(df_raa)
total_unique_links = df_raa['Extern länk'].nunique()
num_languages = df_raa['Språk'].nunique()
langs_sorted = df_raa['Språk'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 305250
Total unique links: 232112
Number of languages: 59

Languages with most links:
Språk
sv        196969
en         19154
da         16022
de         13839
no         11452
es          9396
fi          6242
ja          4308
eu          4078
nl          3439
nn          3150
id          2550
zh          2129
ro          2005
gl          1707
ast         1394
af          1118
an          1097
simple       509
vi           455
lb           405
nan          386
bs           385
pl           362
hak          331
gv           321
ilo          313
cdo          304
ms           296
sco          177
diq          146
tl           127
it           126
frr          104
gpe           56
sc            55
kaa           53
tum           52
tt            48
min           34
sq            33
bcl           22
se            19
ha            17
dtp           14
av            11
su            10
ig            10
ceb            5
nds            3
ace            3
so             2
bar        

In [20]:
from urllib.parse import urlparse
import pandas as pd

# Extract domains
df_raa['domain'] = df_raa['Extern länk'].apply(lambda u: urlparse(u).netloc)

# Count links grouped by domain
links_per_domain = df_raa['domain'].value_counts()

print("Links per domain:")
print(links_per_domain.to_string())


Links per domain:
domain
kulturarvsdata.se                    192018
kulturnav.org                         89497
app.raa.se                             9332
www.bebyggelseregistret.raa.se         3679
bebyggelseregistret.raa.se             3256
www.raa.se                             3106
www.fmis.raa.se                        2164
www.kringla.nu                          828
samla.raa.se                            638
pub.raa.se                              156
kmb.raa.se                              118
www.kms.raa.se                          108
www.k-blogg.se                           87
kringla.nu                               71
www.digisam.se                           41
k-blogg.se                               32
hildebrand.raa.se                        24
minnen.se                                23
www.platsr.se                            15
raa.se                                   14
digisam.se                               12
old.raa.se                                7
fmis.ra

In [21]:
import pandas as pd

# --- Stats ---
total_links = len(df_raa)
total_unique_links = df_raa['Extern länk'].nunique()
num_languages = df_raa['Språk'].nunique()
langs_sorted = df_raa['Språk'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 305250
Total unique links: 232112
Number of languages: 59

Languages with most links:
Språk
sv        196969
en         19154
da         16022
de         13839
no         11452
es          9396
fi          6242
ja          4308
eu          4078
nl          3439
nn          3150
id          2550
zh          2129
ro          2005
gl          1707
ast         1394
af          1118
an          1097
simple       509
vi           455
lb           405
nan          386
bs           385
pl           362
hak          331
gv           321
ilo          313
cdo          304
ms           296
sco          177
diq          146
tl           127
it           126
frr          104
gpe           56
sc            55
kaa           53
tum           52
tt            48
min           34
sq            33
bcl           22
se            19
ha            17
dtp           14
av            11
su            10
ig            10
ceb            5
nds            3
ace            3
so             2
bar        

In [22]:
from datetime import date
import os

# Sätt datum
today = date.today().strftime("%Y_%m_%d")

# Se till att katalogen finns
os.makedirs("results", exist_ok=True)

# Bygg filnamn
outfile = f"results/links_raa_{today}.csv"

# Exportera
df_raa.to_csv(outfile, index=False, encoding="utf-8")

print(f"[OK] Exported {len(df_raa)} rows to {outfile}")


[OK] Exported 305250 rows to results/links_raa_2025_12_01.csv


In [23]:
print(df_raa.columns.tolist())

['Språk', 'Wikipedia-sida', 'Extern länk', 'Wikipedia-länk', 'Domän', 'lang_code', 'lang_name', 'domain']


In [24]:
from urllib.parse import urlparse
import pandas as pd

# Extract domain
df_raa['domain'] = df_raa['Extern länk'].apply(lambda u: urlparse(u).netloc)

# Count links grouped by domain
links_per_domain = df_raa['domain'].value_counts()

print("Links per domain:")
print(links_per_domain.to_string())


Links per domain:
domain
kulturarvsdata.se                    192018
kulturnav.org                         89497
app.raa.se                             9332
www.bebyggelseregistret.raa.se         3679
bebyggelseregistret.raa.se             3256
www.raa.se                             3106
www.fmis.raa.se                        2164
www.kringla.nu                          828
samla.raa.se                            638
pub.raa.se                              156
kmb.raa.se                              118
www.kms.raa.se                          108
www.k-blogg.se                           87
kringla.nu                               71
www.digisam.se                           41
k-blogg.se                               32
hildebrand.raa.se                        24
minnen.se                                23
www.platsr.se                            15
raa.se                                   14
digisam.se                               12
old.raa.se                                7
fmis.ra

In [25]:
GITHUB_REPO = "https://github.com/salgo60/Svenskaforsamlingar"
script_name = SCRIPT_NAME

In [26]:
script_name

'RAA42_linkroot.ipynb'

In [27]:
# --- Stats ---
total_links = len(df_raa)
total_unique_links = df_raa["Extern länk"].nunique()
langs_with_hits = sorted(df_raa["Språk"].unique())

num_languages_found = len(langs_with_hits)
num_languages_checked = len(df_wiki)        # alla språk som genomsöktes
num_languages_found = df_raa['Språk'].nunique()

In [28]:
from pathlib import Path
from datetime import date
import pandas as pd

def save_sortable_html_df_raa(
    df,
    out_dir="results",
    domains=None,
    issue_url="https://github.com/salgo60/SamlaLibris/issues/42",
):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True)

    today = date.today().strftime("%Y_%m_%d")
    out_path = out_dir / f"links_raa_{today}.html"
    rerun_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    domains = domains or [] 

    # Gör länkar klickbara om de finns
    df = df.copy()  
    if "Wikipedia-länk" in df.columns:
        df["Wikipedia-länk"] = df["Wikipedia-länk"].apply(
            lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
        )
    if "Extern länk" in df.columns:
        df["Extern länk"] = df["Extern länk"].apply(
            lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
        )

    html_table = df.to_html(
        classes="pivot",
        border=0,
        escape=False,   # viktigt för klickbara länkar
        index=False
    )
   
    css = """
    <style>
      body {
        font-family: Arial, sans-serif;
        margin: 20px;
      }
      table.pivot {
        border-collapse: collapse;
        width: 100%;
        font-size: 12px;
      }
      table.pivot th, table.pivot td {
        border: 1px solid #999;
        padding: 6px 8px;
        text-align: left;
        vertical-align: top;
        white-space: normal;
      }
      table.pivot th {
        cursor: pointer;
        background: #f2f2f2;
      }
      table.pivot th:hover {
        background: #e2e2e2;
      } 
      table.pivot thead th {
          position: sticky;
          top: 0;
          background: #f2f2f2;
          z-index: 2;
        }
        
        /* Sorting indicators */
        table.pivot th::after {
          content: "";
          float: right;
          opacity: 0.4;
        }
        
        table.pivot th.sorted-asc::after {
          content: " ▲";
        }
        
        table.pivot th.sorted-desc::after {
          content: " ▼";
        }
          .meta {
            background: #f8f8f8;
            border: 1px solid #ccc;
            padding: 12px;
            margin-bottom: 20px;
            font-size: 13px;
          }
          .meta h2 {
            margin-top: 0;
          }
          .meta ul {
            margin: 0;
            padding-left: 18px;
          }
    </style>
    """

    js = """
    <script>
    document.addEventListener('DOMContentLoaded', () => {
        document.querySelectorAll("table.pivot th").forEach((header, colIndex) => {
            header.addEventListener("click", () => {
                const table = header.closest("table");
                const tbody = table.querySelector("tbody");
                const rows = Array.from(tbody.querySelectorAll("tr"));
                const asc = !header.classList.contains("sorted-asc");

                rows.sort((a, b) => {
                    const A = a.children[colIndex].innerText.trim();
                    const B = b.children[colIndex].innerText.trim();

                    const numA = parseFloat(A.replace(",", ".")) || null;
                    const numB = parseFloat(B.replace(",", ".")) || null;

                    if (numA !== null && numB !== null) {
                        return asc ? numA - numB : numB - numA;
                    }
                    return asc ? A.localeCompare(B) : B.localeCompare(A);
                });

                table.querySelectorAll("th").forEach(th =>
                    th.classList.remove("sorted-asc","sorted-desc")
                );
                header.classList.add(asc ? "sorted-asc" : "sorted-desc");

                rows.forEach(row => tbody.appendChild(row));
            });
        });
    });
    </script>
    """
    meta_html = f"""
    <div class="meta">
      <h2>Metadata</h2>
    
      <p><strong>Rerun:</strong> {rerun_ts}</p>
      <p><strong>Script:</strong>
         <a href="{SCRIPT_URL}" target="_blank">{SCRIPT_NAME}</a>
      </p>
      <p><strong>Issue:</strong>
         <a href="{issue_url}" target="_blank">{issue_url.split('/')[-1]}</a>
      </p>
    
      <p><strong>Datakällor:</strong></p>
      <div class="meta-indent">
        Wikidata<br>
        MediaWiki API – exturlusage
      </div>
    
      <p><strong>Parametrar:</strong></p>
      <div class="meta-indent">
        Språkfilter: exkluderar "special"
      </div>
    
      <p><strong>Antal språk genomsökta:</strong> {num_languages_checked}</p>
      <p><strong>Antal språk med träffar:</strong> {num_languages_found}</p>
      <p><strong>Wikipedia-språk med träffar:</strong> {", ".join(langs_with_hits)}</p>
      <p><strong>Totala länkar:</strong> {total_links}</p>
      <p><strong>Unika länkar:</strong> {total_unique_links}</p>
    </div>
    """
    
    html = f"""
    <html>
    <head>
      <meta charset="utf-8">
      <title>RAÄ links in Wikipedia</title>
      {css}
    </head>
    <body>
      <h1>Wikipedia → RAÄ</h1>
      {meta_html}
      <p>Sorterbar tabell. Klicka på kolumnrubriker för sortering.</p>
      {html_table}
      {js}
    </body>
    </html>
    """

    out_path.write_text(html, encoding="utf-8")
    print(f"✅ HTML skapad: {out_path}")


In [29]:
save_sortable_html_df_raa(df_raa)


✅ HTML skapad: results/links_raa_2025_12_01.html


In [30]:
df_raa.head()

,Språk,Wikipedia-sida,Extern länk,Wikipedia-länk,Domän,lang_code,lang_name,domain
0,ace,Leonard Cohen,http://kulturnav.org/language/en/7e908441-ad8d...,https://ace.wikipedia.org/wiki/Leonard_Cohen,kulturnav.org,ace,Acèh,kulturnav.org
1,ace,Agnetha Fältskog,http://kulturnav.org/language/en/1718a400-a2bc...,https://ace.wikipedia.org/wiki/Agnetha_Fältskog,kulturnav.org,ace,Acèh,kulturnav.org
2,ace,Anni-Frid Lyngstad,http://kulturnav.org/language/en/a75e48ae-e70f...,https://ace.wikipedia.org/wiki/Anni-Frid_Lyngstad,kulturnav.org,ace,Acèh,kulturnav.org
3,af,Adils,http://www.raa.se/cms/extern/se_och_besoka/sev...,https://af.wikipedia.org/wiki/Adils,raa.se,af,Afrikaans,www.raa.se
4,af,Oscar Niemeyer,http://kulturnav.org/6aa00405-f813-4e04-96b2-1...,https://af.wikipedia.org/wiki/Oscar_Niemeyer,kulturnav.org,af,Afrikaans,kulturnav.org


### Domains per language 

In [31]:
# 1. Pivot table: domains per language
domain_by_lang = df_raa.pivot_table(
    index='Språk',
    columns='domain',
    values='Extern länk',
    aggfunc='count',
    fill_value=0
)

print("Links per domain for each language:")
print(domain_by_lang)

# 2. Sorted list of (lang, domain)
lang_domain_counts = (
    df_raa.groupby(['Språk', 'domain'])
      .size()
      .reset_index(name='count')
      .sort_values('count', ascending=False)
)

print("\nTop language-domain combinations:")
print(lang_domain_counts)


Links per domain for each language:
domain  agendakulturarv.raa.se  app.raa.se  bebyggelseregistret.raa.se  \
Språk                                                                    
ace                          0           0                           0   
af                           0           0                           0   
an                           0           0                           0   
ast                          0           1                           0   
av                           0           0                           0   
bar                          0           0                           0   
bcl                          0           0                           0   
bs                           0           0                           0   
cdo                          0           0                           0   
ceb                          0           0                           0   
da                           0           5                           5   
de

In [32]:
from pathlib import Path
import pandas as pd

def save_sortable_html(df, out_path):
    out_path = Path(out_path)

    html_table = df_raa.to_html(
        classes="pivot",
        border=0,
        na_rep="0"
    )

    css = """
    <style>
      body {
        font-family: Arial, sans-serif;
        margin: 20px;
      }
      table.pivot {
        border-collapse: collapse;
        width: 100%;
        font-size: 12px;
      }
      table.pivot th, table.pivot td {
        border: 1px solid #999;
        padding: 6px 8px;
        text-align: right;
        white-space: normal;       /* <-- låter header visa hela texten */
      }
      table.pivot th {
        cursor: pointer;            /* <-- visar att header är klickbar */
        background: #f2f2f2;
      }
      table.pivot th:hover {
        background: #e2e2e2;
      }
    </style>
    """

    js = """
    <script>
    // Enkel tabellsortering i ren JS
    document.addEventListener('DOMContentLoaded', () => {
        document.querySelectorAll("table.pivot th").forEach((header, colIndex) => {
            header.addEventListener("click", () => {
                const table = header.closest("table");
                const tbody = table.querySelector("tbody");
                const rows = Array.from(tbody.querySelectorAll("tr"));
                const asc = !header.classList.contains("sorted-asc");

                // Sortera
                rows.sort((a, b) => {
                    const A = a.children[colIndex].innerText.trim();
                    const B = b.children[colIndex].innerText.trim();
                    const numA = parseFloat(A.replace(",", ".")) || 0;
                    const numB = parseFloat(B.replace(",", ".")) || 0;
                    return asc ? numA - numB : numB - numA;
                });

                // Rensa ev tidigare sort-klass
                table.querySelectorAll("th").forEach(th => th.classList.remove("sorted-asc","sorted-desc"));

                // Sätt sorteringsklass för stil
                header.classList.add(asc ? "sorted-asc" : "sorted-desc");

                // Stoppa tillbaka raderna i sorterad ordning
                rows.forEach(row => tbody.appendChild(row));
            });
        });
    });
    </script>
    """

    html = f"""
    <html>
    <head>
    <meta charset="utf-8">
    <title>Sortable Pivot Table RAÄ</title>
    {css}
    </head>
    <body>
    <h1>Domain-by-language matrix</h1>
    <p>Klicka på valfri kolumnheader för sortering.</p>
    {html_table}
    {js}
    </body>
    </html>
    """

    out_path.write_text(html, encoding="utf-8")
    print("HTML skapad:", out_path)

# Skriv HTML-fil
save_sortable_html(domain_by_lang, "results/domain_by_lang_raa.html")


HTML skapad: results/domain_by_lang_raa.html


In [33]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-12-01 18:40:36
Total time elapsed: 13 minutes 46.33 seconds
